# Using Genestack Omics APIs for query expression values

### Set instance and token

In [7]:
library(integrationCurator)
library(expressionUser)
library(stringi)
suppressMessages(library(tidyverse))
suppressMessages(library(integrationCurator)) # Genestack client library

Sys.setenv(PRED_SPOT_HOST = 'br-dev.genestack.net',
           PRED_SPOT_TOKEN = '<token>',
           PRED_SPOT_VERSION = 'default-released')

### Get samples by study accession

In [8]:
study_acc = "GSF129969"
samples <- OmicsQueriesApi_search_samples(
  study_filter=paste('genestack:accession=', study_acc, sep=""),
  sample_filter=''
)$content$data[['metadata']]
head(samples)

,genestack:accession,Sample Source ID,Sample Name,Organism,Sex,Disease,Age,Age Unit,Tissue,Cell Type,Cell Line,Compound Treatment / Compound,Compound Treatment / Dose,Compound Treatment / Dose Unit,Data Files / Raw,Data Files / Processed,Import Source URL,Sample Source,Population,groupId
,<chr>,<chr>,<lgl>,<lgl>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>
1,GSF129971,SRR6441195,NA,NA,M,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1000 Genomes Project,British,GSF129970
2,GSF129972,SRR6441188,NA,NA,F,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1000 Genomes Project,British,GSF129970
3,GSF129973,SRR6441196,NA,NA,M,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1000 Genomes Project,Finnish,GSF129970
4,GSF129974,SRR6441197,NA,NA,F,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1000 Genomes Project,Finnish,GSF129970


In [9]:
# integration/link/expression/group/by/study/
expression_group_acc <- ExpressionIntegrationApi_get_parents_by_study(id=study_acc)$content$itemId
expression_group_acc

[1] "GSF129982"

In [10]:
# integration/link/expression/run-to-samples/by/group
run_to_samples <- ExpressionIntegrationApi_get_run_to_sample_pairs(id=expression_group_acc)$content$data
head(run_to_samples)

,runId,sampleId
,<chr>,<chr>
1,19286,GSF129977
2,19287,GSF129978
3,19288,GSF129979


In [16]:
# /expression
feature_list = "ENSG00777"
run_list <- stri_join_list(run_to_samples["runId"], sep=",")
res_expr <- ExpressionSPoTApi_get_expression_data(run_filter=run_list, feature_list=feature_list)$content
# cursor <- res_expr$cursor
data <- res_expr$data
my_data <- as_tibble(data)
my_data %>% select(runId, gene, expression)

runId,gene,expression
<chr>,<chr>,<dbl>
19286,ENSG00777,21.99060
19287,ENSG00777,19.99062
19288,ENSG00777,21.31963


In [17]:
sample_expressions = left_join(run_to_samples, my_data, by=c("runId"="runId"))
sample_expressions %>% select(sampleId, expression)

sampleId,expression
<chr>,<dbl>
GSF129977,21.99060
GSF129978,19.99062
GSF129979,21.31963
